**Word2Vec** é um método para gerar *Word Embeddings* a partir de um corpus de texto, utilizando redes neurais.

Desenvolvido por Tomas Mikolov *et al.* (Google) em 2013, é um dos métodos de geração de *word embeddings* mais populares em tarefas de processamento de linguagem natural (PLN) como análise de sentimento, tradução de textos e reconhecimento de entidades nomeadas (NER).


### Passo 1 - importanto as bibliotecas
Vamos primeiro instalar e importar as bibliotecas que utilizaremos.

In [7]:
import re
import numpy as np
from gensim import corpora, models, similarities
import nltk
import pickle
import pandas as pd
import unicodedata
import spacy 

Para o pré-processamento em língua portuguesa, vamos usar o pacote ```pt_core_news_sm```. Instale antes com:
    
```python -m spacy download pt_core_news_sm```

In [8]:
nlp = spacy.load('pt_core_news_sm') 

# Passo 2 - Pré-processamento do *corpus*


Vamos abrir o arquivo e transformá-lo em um *dataframe*.

In [9]:
file = open(r"corpus_scielo.txt", "r", encoding='UTF-8')
df = pd.DataFrame(file)
df.columns = ['lines']
df = df.sort_index()
file.close()

Assim são os textos.

In [10]:
df

,lines
0,A avifauna da região de Vila Bela da Santíssim...
1,Composição de bandos mistos de aves em fragmen...
2,"Morcegos do Estado do Paraná, Brasil (Mammalia..."
3,Riqueza da fauna de formigas (Hymenoptera: For...
4,Lista de tipos de Dermaptera e Orthoptera (Ins...
...,...
729650,Antonio Austregésilo ou a grandeza e a dignida...
729651,A 20.a enfermaria de Santa Casa em 1920.\n
729652,"Antonio Austregésilo, o psiquiatra.\n"
729653,As reações de Takata e formol-gel em face da p...


Vamos primeiro remover os acentos;

In [11]:
def remove_accents(text):
    '''Strip accents out.'''
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

clean2 = lambda x: cleaning1(x)

In [12]:
df = pd.DataFrame(df.lines.apply(remove_accents))

In [13]:
df

,lines
0,A avifauna da regiao de Vila Bela da Santissim...
1,Composicao de bandos mistos de aves em fragmen...
2,"Morcegos do Estado do Parana, Brasil (Mammalia..."
3,Riqueza da fauna de formigas (Hymenoptera: For...
4,Lista de tipos de Dermaptera e Orthoptera (Ins...
...,...
729650,Antonio Austregesilo ou a grandeza e a dignida...
729651,A 20.a enfermaria de Santa Casa em 1920.\n
729652,"Antonio Austregesilo, o psiquiatra.\n"
729653,As reacoes de Takata e formol-gel em face da p...


Agora, vamos limpar, usando expressões regulares, tudo que não é caracter alfanumérico e passar tudo para caixa baixa.


In [14]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['lines'])

In [15]:
brief_cleaning

<generator object <genexpr> at 0x000002B77A4794A0>

Vamos lematizar e retirar as *stopwords*, para diminuir a dimensionalidade, com a biblioteca ```spacy```. Vamos usar ```spacy.pipe()```.

In [16]:
def cleaning(doc):
    # lematizar e retirar stopwords
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # como Word2Vec usa palavras de contexto para aprender a representação vetorial de uma palavra-alvo,
    # se uma frase tiver apenas uma ou duas palavras,
    # o benefício para o treinamento é muito pequeno    
    if len(txt) > 2:
        return ' '.join(txt)

In [17]:
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_process=-1)]

In [19]:
txt[68162:68167]

['proteinas ryr ryr apresentar papar distinto mecanismo excitacao contracao diferencas mecanismo ativacao respostar ligante',
 '  rna total filar peitar pectoralis major m pse l h gt ph nao pse lt l h gt linhagem distinto cortar posturar utilizar estudar expressao genica gene ryr ryr pcr real',
 '  valorar medios expressao genica relativo rq inferior p lt frango pse linhagem comparar frango nao pse',
 '  outro nao haver diferencas p gt expressao independentemente linhagem estudar',
 '  resultar rq ryr indicar amostrar pse alteracao proporcao ryr ryr comumente encontrar musculos ave']

In [20]:
txt2 = [str(row.strip()) for row in txt if (row != None and row.strip() != '.')]

In [21]:
txt2[63263:63268]

['resultar mostrar melhor co solvente extracao tanino co supercritico aguar concentracao melhor solvente lavagem trap metanol',
 'degradacao herbicida diuron estudar solo historico aplicacao obter aproximadamente degradacao solo historico aplicacao apo dia incubacao',
 'haver aumentar numerar bacterias solo historico aplicacao x x ufc g solo',
 'nao haver aumentar biomassa apo incubacao pôr encontrar significante residuo c diuron biomassa',
 'consorciar tres bacterias isolar acinetobacter johnsonii especies bacillus sp conter diuron unica fonte carbono']

Vamos remover os valores faltantes e duplicados.

In [22]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean

,clean
0,avifauna regiao vila belo santissima trindade ...
1,composicao bando misto ave fragmento matar atl...
2,morcego parana brasil mammalia chiroptera riqu...
3,riqueza fauna formigo hymenoptera formicidae h...
4,lista tipo dermaptera orthoptera insecta depos...
...,...
729645,orientacao profissional importancia
729650,antonio austregesilo grandeza dignidade espiritar
729651,enfermar santo casar
729652,antonio austregesilo psiquiatro


In [23]:
len(txt2)

724700

#### Bigramas
O pacote ```Gensim Phrases``` detecta automaticamente frases comuns (bigramas) de uma lista de frases.

Veja mais em: https://radimrehurek.com/gensim/models/phrases.html

Com o método ```Phrases()```, vamos criar frases relevantes a partir da lista de frases.


In [24]:
from gensim.models.phrases import Phrases, Phraser

sent = [row.split() for row in df_clean['clean']]

In [25]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

```Phraser()``` reduz o consumo de memória de ```phrases``` ao descartar estado do modelo não necessário para a detecção de bigramas.

In [26]:
bigram = Phraser(phrases)

Transformamos o corpus com base nos bigramas detectados.

In [27]:
sentences = bigram[sent]

#### Palavras frequentes
Vamos calcular a frequencia das palavras, para verificar a eficácia da lematização, remoção de palavras irrelevantes e adição de bigramas.

In [28]:
from collections import defaultdict 

word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

148119

In [29]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['paciente',
 'resultar',
 'estudar',
 'nao',
 'apresentar',
 'analisar',
 'ser',
 'realizar',
 'utilizar',
 'ano']

### Passo 3 - Treinamento do modelo

Hora de treinar nosso modelo Word2Vec com nossos dados. Primeiro, vamos verificar o ambiente.

In [30]:
import multiprocessing
from gensim.models import Word2Vec
from time import time

cores = multiprocessing.cpu_count() # conta o número de núcleos do computador

Vamos parametrizar nosso modelo.

- *min_count*: Ignora todas as palavras com frequência absoluta total inferior a esta
- *window*: A distância máxima entre a palavra atual e a prevista em uma frase. Por exemplo, palavras da janela à esquerda e palavras da janela à esquerda do nosso alvo
- *size*: Dimensionalidade dos vetores
- *sample*: O limite para configurar quais palavras de alta frequência são reduzidas aleatoriamente
- *alpha*: A taxa de aprendizagem inicial
- *min_alpha*: A taxa de aprendizado cairá linearmente para *min_alpha* conforme o treinamento progride
- *negative*: Se > 0, a amostragem negativa será usada, o int para negativo especifica quantas "palavras de ruído" devem ser eliminadas. Se definido como 0, nenhuma amostra negativa é usada
- *workers*: Quantidade de *threads* de trabalho para treinar o modelo (treinamento mais rápido com máquinas multicore)

In [31]:
w2v_model = Word2Vec(min_count=3,
                     window=2,
                     vector_size=32,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=10,
                     workers=cores-1)

#### Construindo o vocabulário e treinando o modelo
Parâmetros do treinamento:

*total_examples: Contagem de sentenças;
*epochs*: Número de iterações (épocas) no corpus

In [32]:
w2v_model.build_vocab(sentences, progress_per=10000)


In [33]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


(158162476, 230529330)

### Passo 4 - Salvando o modelo
Vamos salvar o modelo nos formatos *KeyedVectors* e binário, para utilizarmos posteriormente.

In [34]:
w2v_model.init_sims(replace=True)  # deixa o modelo mais eficiente, pois não vamos mais treiná-lo futuramente

<ipython-input-34-bc56ad85d6ac>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)  # deixa o modelo mais eficiente, pois não vamos mais treiná-lo futuramente


In [35]:
w2v_model.save("corpus_scielo.model")
w2v_model.wv.save_word2vec_format('corpus_scielo.bin', binary=True)



Agora que já treinamos nosso modelo, [vamos ver aqui como utilizá-lo](https://github.com/lisaterumi/word2vec-harry-potter-portugues/blob/main/%5B2%5D%20tSNE-Harry-Potter.ipynb).